**COMPONENTE DE PROYECTO DE INTEGRACIÓN CURRICULAR**

**PROGRAMA GENERADOR DE DESCRIPCIONES**

Se levanta un servicio dentro de un Docker para que pueda ser consumido por un programa externo

**Instalación de dependencias para modelos de Embeddings y fragmentación de texto - RAG**

In [ ]:
# Librería para transformar oraciones en vectores de datos
!pip install sentence-transformers
# Librería para realizar comparaciones y buscar las mejores coincidencias de
# vectores de datos
!pip install faiss-cpu
# Librería para la extracción y manipulación de texto en archivos LaTeX
!pip install pylatexenc

**Importación de Librerías**

In [11]:
# Importación de librerías para consumo de la API de Oobabooga, para enviar
# peticiones al modelo Llama 2 70B
import requests
import json
# Importación de librerías para LaTeX y expresiones regulares.
import re
from pylatexenc.latex2text import LatexNodes2Text
# Importación de librerías para generar embeddings de texto con el modelo all-MiniLM-L6-v2
from sentence_transformers import SentenceTransformer
import nltk
import numpy as np
# Importación de la libreria faiss para generar un Indexer
import faiss
# Importación de la librería pandas para generar dataframes
import pandas as pd
# Importación de librerías para levantar el servicio del programa generador de descripciones
from flask import Flask, request
import os
# Importación de librería para extraer el contenido de un archivo .zip
import zipfile

In [33]:
# Descargar recursos de nltk (solo es necesario la primera vez)
nltk.download('punkt')
# Cargar el modelo de embeddings
embeddingModel = SentenceTransformer('all-MiniLM-L6-v2')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**APLICACIÓN DE TÉCNICA RETRIEVAL AUGMENTED GENERATION (RAG)**

Funciones de procesamiento del Texto Latex

In [30]:
# Función para extraer el código de una figura en archivo externo
def extract_code_figure(figFile):
    with open(figFile, 'r') as file:
        document = file.read()
    return document

# Función para alinear el texto y eliminar saltos de línea vacíos
def remove_empty_lines_and_align_text(text):
    # Eliminar los saltos de línea donde no haya texto
    text = re.sub(r'\n\s*\n', '\n', text)
    # Se eliminan las tabulaciones que tengan las líneas de texto al inicio
    lines = text.splitlines()
    aligned_lines = []
    for line in lines:
        if line:
            aligned_lines.append(line.lstrip())
    return "\n".join(aligned_lines)

# Función para eliminar líneas comentadas
def remove_commented_lines(content):
    lines = content.split('\n')
    uncommented_lines = [line for line in lines if not line.strip().startswith('%')]
    return '\n'.join(uncommented_lines)

# Función para leer el contenido de un archivo
def read_input_file(file_path):
  # Lista de extensiones posibles
  possible_extensions = ['.tikz', '.tex', '.svg']  # Añade aquí todas las extensiones posibles
  for ext in possible_extensions:
    full_path = file_path + ext
    #print(full_path)
    if os.path.exists(full_path):
      with open(full_path, 'r') as file:
        return file.read()
  return None

# Función para extraer el contenido de las figuras del documento .tex
# Y también para extraer el texto del documento
def extract_figures_and_text(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Eliminar líneas comentadas
    uncommented_content = remove_commented_lines(content)

    # Eliminar macros \href antes de la conversión
    uncommented_content = re.sub(r'\\href\{[^\}]*\}\{[^\}]*\}', '', uncommented_content)

    # Extraer todas las figuras
    figures = re.findall(r'\\begin\{figure\}(.*?)\\end\{figure\}', uncommented_content, re.DOTALL)

    # Procesar cada figura para extraer el nombre del archivo, el caption y el label
    figure_data = []

    # Obtener el directorio del archivo
    directory_path = os.path.dirname(file_path)

    global code_Figures
    contFig = 0
    for figure in figures:
        # Diccionarios para los datos de las Figuras y para sus códigos
        data = {}
        code_filename_match = re.search(r'\\includesvg(?:\[.*?\])?\{([^}]*)\}', figure)
        filename_match = re.search(r'\\includegraphics(?:\[.*?\])?\{([^}]*)\}', figure)
        caption_match = re.search(r'\\caption\{([^}]*)\}', figure)
        label_match = re.search(r'\\label\{([\w:\s*-]+)\}', figure)
        # Para figuras con código SVG en archivo externo
        if code_filename_match:
            contFig += 1
            data['Figure'] = contFig
            data['format'] = "SVG"
            data['filename'] = code_filename_match.group(1)
            # Se guarda el código de la figura
            data['code'] = extract_code_figure(f'{directory_path}/{data["filename"]}')
        # Para figuras con código tikz en archivo externo
        elif filename_match:
            # Obtener la extensión del archivo
            _, extension = os.path.splitext(filename_match.group(1))
            if extension in ['.tikz']:
              contFig += 1
              data['Figure'] = contFig
              data['format'] = "TikZ"
              data['filename'] = filename_match.group(1)
              data['code'] = extract_code_figure(f'{directory_path}/{data["filename"]}')
            else:
              break
        # Para figuras con código Tikz embebido en el documento
        else:
          # Encuentra todas las etiquetas \input
          inputs = re.findall(r'\\input\{(.*?)\}', figure)
          #print(inputs)
          if inputs:
            for input_file in inputs:
              input_content = read_input_file(f'{directory_path}/{input_file}')
              # Reemplaza la etiqueta \input con el contenido del archivo
              figure = figure.replace(f'\\input{{{input_file}}}', input_content)
          contFig += 1
          data['Figure'] = contFig
          data['format'] = "Tikz or SVG file"
          data['code'] = figure

        if caption_match:
            data['caption'] = caption_match.group(1)
        if label_match:
            data['label'] = label_match.group(1)
        figure_data.append(data)

    # Crear un diccionario de información de figuras (incluyendo figuras sin etiquetas)
    figure_info = {}
    for data in figure_data:
      if 'filename' in data:
        figure_info[data['label']] = (data['filename'], data['caption'])
      else:
        # Asignar el número de Figura para las figuras sin archivo externo
        figure_info[data['label']] = (data['Figure'], data['caption'])

    # Reemplazar las referencias a figuras en el contenido original
    uncommented_content = re.sub(r'Figure(?:\s*[:~])?\s*\\ref\{(.+?)\}', lambda match: f"Figure named:\"{figure_info.get(match.group(1), ('', ''))[0]}\" and label:\"{match.group(1)}\"", uncommented_content)

    # Obtener el contenido sin las figuras
    content_without_figures = re.sub(r'\\begin\{figure\}.*?\\end\{figure\}', '', uncommented_content, flags=re.DOTALL)

    # Obtener el título del documento
    title_patern = r'\\title(?:\[[^\]]*\])?\{([^\}]*)\}'
    title_match = re.search(title_patern, content_without_figures, re.DOTALL)
    if title_match:
      # Variable global para guardar el titulo del documento
      global title
      title = title_match.group(1)

    # Encuentra el contenido entre \title y \begin{document}
    patron = r'\\title(?:\[[^\]]*\])?\{([^\}]*)\}.*?\\begin\{document\}(.*?)\\end\{document\}'
    coincidencias = re.search(patron, content_without_figures, re.DOTALL)

    if coincidencias:
      titulo = coincidencias.group(1)
      contenido_documento = coincidencias.group(2)
      # Se une en un solo pre documento
      pre_text = f"{titulo}\n\n{contenido_documento}"
      # Convertir el contenido restante a texto
      l2t = LatexNodes2Text()
      text = l2t.latex_to_text(pre_text)
    else:
      print("No se encontró contenido entre \\title y \\begin{document}.")
      # Convertir el contenido restante a texto
      l2t = LatexNodes2Text()
      text = l2t.latex_to_text(content_without_figures)

    # Se alinea el texto
    text = remove_empty_lines_and_align_text(text)

    return figure_data, text

**Fragmentación del documento y conversión a Embeddings**

In [14]:
 # Función para fragmentar y obtener los embeddings del archivo
def fragment_and_get_embeddings(file_path):
  # Se llama a la función que extrae el texto y la información de las figuras
  figure_data, text = extract_figures_and_text(file_path)
  # Se divide el texto en oraciones
  sentences = nltk.sent_tokenize(text)
  # Se generan los embeddings de las oraciones
  sentence_embeddings = embeddingModel.encode(sentences)
  return sentence_embeddings, sentences, figure_data

**Indexación de los embeddings**

In [15]:
def indexing(embeddings):
  # Crear índice de tpo FlatL2 y almacenar embeddings
  index = faiss.IndexFlatL2(embeddingModel.get_sentence_embedding_dimension())
  index.add(embeddings)
  return index

**Definición de funciones para consumir el modelo Llama2**

In [16]:
# Función básica de consumo del modelo - no contextualizado
def generate_text_basic(prompt):
    url = "http://localhost:5000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    payload = {
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "mode": "instruct",
        "instruction_template": "Alpaca"
    }
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        #result = response.json()["result"]["response"]
        result = response.json()["choices"][0]["message"]["content"]
        return result
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")

In [17]:
# Función de consumo del modelo con generación mediante RAG
def generate_text_rag(prompt, index, sentences):

    global conversation_history
    url = "http://172.19.0.2:5000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    # Condicionar al modelo con el rol de system cada que haya una nueva conversación
    if len(conversation_history) == 0 :
        conversation_history.append({"role": "system", "content": "You are a helpful assistant and your job is to improve descriptions of figures. You will only aswer the description of the figures given. No more."})

    # Codificar el prompt
    prompt_embedding = embeddingModel.encode([prompt])[0]

    # Buscar las oraciones más relevantes en el índice
    distances, indices = index.search(prompt_embedding.reshape(1, -1), 5)
    relevant_sentences = [sentences[idx] for idx in indices[0]]
    #print(relevant_sentences)

    # Agregar las oraciones relevantes al historial de la conversación
    conversation_history.append({"role": "system", "content": "\n".join(relevant_sentences)})

    # Agregar el prompt actual a la lista de historial
    conversation_history.append({"role": "user", "content": prompt})

    payload = {
        "messages": conversation_history,
        "mode": "instruct",
        "instruction_template": "Alpaca"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        result = response.json()["choices"][0]["message"]["content"]

        # Agregar la respuesta del modelo al historial de la conversación
        conversation_history.append({"role": "assistant", "content": result})

        return result
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")

In [18]:
# Función consumir el modelo y enviar la figura códificada
def generate_text_adv(prompt):

    global conversation_history
    url = "http://localhost:5000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    # Agregar el prompt actual a la lista de historial
    conversation_history.append({"role": "user", "content": prompt})

    payload = {
        "messages": conversation_history,
        "mode": "instruct",
        "instruction_template": "Alpaca"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        result = response.json()["choices"][0]["message"]["content"]

        # Agregar la respuesta del modelo al historial de la conversación
        conversation_history.append({"role": "assistant", "content": result})

        return result
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")

**Generación de respuestas RAG**

In [10]:
# Función para la generación de descirpciones
def figures_descriptor(file_path):
  global conversation_history
  global code_Figures
  # Se inicializa el historial de la conversación
  conversation_history = []
  # Variable globarl para guardar el código de las figuras
  code_Figures = []
  # Se obtienen los embeddings de las oraciones
  embeddings, sentences, figures_colec = fragment_and_get_embeddings(file_path)
  # Se indexa los embeddings
  index = indexing(embeddings)
  # Se genera un bucle que recorra cada Figura
  for figure in figures_colec:
    if 'filename' in figure:
      figure_name = figure['filename']
    else:
      figure_name = figure['Figure']

    if figure['format'] == "EPS":
      figure['description_final'] = figure['caption']
    else:
      # Envió y respuesta de la petición 1
      prompt_basic_1 = f"I need you to describe a figure based on the following figure code: {figure['code']}. Reflect and process this request twice to generate the description. Just respond with the description, nothing more."
      description_base = generate_text_adv(prompt_basic_1)
      #print(description_basic)
      # Se agrega la descripción base a la Figura
      figure['description_base'] = description_base

      # Envió y respuesta de la petición 2
      prompt_basic = f"Previous figure appears in the document titled {title}, with name '{figure_name}', with label '{figure['label']}' and with caption {figure['caption']}. Describe it as best as possible based only on the information given you can identify. Just respond with the description, nothing more."
      description_rag = generate_text_rag(prompt_basic, index, sentences)
      #print(description_basic)
      # Se agrega la descripción base a la Figura
      figure['description_advance'] = description_rag

      # Envió y respuesta de la petición 3
      prompt_final = f"From the last description you gave me, try to describe the equations, if they exist, and all the details of the figure in a way that the figure can be understood without seeing it. Just respond with the description. nothing more."
      description_final = generate_text_adv(prompt_final)
      # Se agrega la descripción base a la Figura
      figure['description_final'] = description_final

      # Se reinicializa el historial de la conversación
      conversation_history = []
  return figures_colec

**PROGRAMA PRINCIPAL (MAIN)**

Función para obtener y descomprimir el archivo .zip

In [ ]:
def upload_file(file):
    if file.endswith('.zip'):
      # Extraer el nombre base del archivo .zip sin la extensión
      zip_name = os.path.splitext(os.path.basename(file))[0]
      # Crear una nueva carpeta con el nombre base en el directorio destino
      zip_path = os.path.join('/media/Llama-MSVH/Archivos_TEX', zip_name)
      os.makedirs(zip_path, exist_ok=True)

      # Se descomprime el .zip para obtener el archivo .tex
      with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(zip_path)

      # Buscar el archivo .tex en el directorio extraído
        tex_file_path = None
        for root, dirs, files in os.walk(zip_path):
            for file in files:
                if file.endswith('.tex'):
                    tex_file_path = os.path.join(root, file)
                    break
            if tex_file_path:
                break

        if not tex_file_path:
            return "No .tex file found in the ZIP archive"

        return 'Archivo ZIP recibido correctamente'
    else:
        return 'Formato de archivo no válido'

Se levanta el servicio para obtener el archivo .ZIP

In [34]:
from itertools import count
from flask import jsonify

# Contador
# count = 0
# Se instancia un objeto Flask para el servicio
app = Flask(__name__)

@app.route('/upload', methods=['POST', 'GET'])
def upload_file():
    # Contador
    count = 0
    file = request.files['file']
    if file.filename.endswith('.zip'):
      # Extraer el nombre base del archivo .zip sin la extensión
      zip_name = os.path.splitext(file.filename)[0]
      # Crear una nueva carpeta con el nombre base en el directorio destino
      zip_path = os.path.join('/media/Llama-MSVH/Archivos_TEX', zip_name)
      os.makedirs(zip_path, exist_ok=True)

      # Guardar el archivo zip en el directorio destino
      file_path = os.path.join(zip_path, file.filename)
      file.save(file_path)
      print(f"Archivo ZIP recibido: {file_path}")

      count += 1
      # Se descomprime el .zip para obtener el archivo .tex
      with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(zip_path)

      # Buscar el archivo .tex en el directorio extraído
        tex_file_path = None
        for root, dirs, files in os.walk(zip_path):
            for file in files:
                if file.endswith('.tex'):
                    tex_file_path = os.path.join(root, file)
                    break
            if tex_file_path:
                break

        if not tex_file_path:
            return "No .tex file found in the ZIP archive", 400

        # Se llama a la función generadora de descripciónes
        figures_desc_file = figures_descriptor(tex_file_path)
        # Crear un DataFrame a partir de la lista de diccionarios
        df = pd.DataFrame(figures_desc_file)
        # Escribir el DataFrame a un archivo XLSX
        df.to_excel('Excel_Desc.xlsx', index=False)
        print("Archivo Num: ")
        print(count)

        # Crear una lista de diccionarios con los atributos deseados
        response_data = []
        for figure in figures_desc_file:
            figure_dict = {
                'Figure': figure['Figure'],
                'format': figure['format'],
                'label': figure['label'],
                'description_final': figure['description_final']
            }
            response_data.append(figure_dict)

        print("Enviando la información...")
        print(jsonify(response_data))
        # Convertir la lista de diccionarios a JSON y devolverla como respuesta
        return jsonify(response_data)
    else:
        return 'Formato de archivo no válido', 400

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8001)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8001
 * Running on http://172.19.0.2:8001
INFO:werkzeug:Press CTRL+C to quit


Archivo ZIP recibido: /media/Llama-MSVH/Archivos_TEX/Paper2/Paper2.zip


ERROR:__main__:Exception on /upload [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 497, in _make_request
    conn.request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 395, in request
    self.endheaders()
  File "

In [ ]:
# Prueba
resp = upload_file('/media/Llama-MSVH/Archivos_TEX/prueba.zip')
print(resp)

Archivo ZIP recibido correctamente


PRUEBA SIN SERVICIO

In [ ]:
# Se llama a la función generadora de descripciónes
figures_desc_file = figures_descriptor("/media/Llama-MSVH/Archivos_TEX/paper1/GGGL2-arxiv.tex")
# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(figures_desc_file)
# Escribir el DataFrame a un archivo XLSX
df.to_excel('Excel_Desc1.xlsx', index=False)

Unexpected but recurrent phenomena for\\ Quot and Hilbert schemes of points
Contenido completo:


In [ ]:
# Se llama a la función generadora de descripciónes
figures_desc_file = figures_descriptor("/media/Llama-MSVH/Archivos_TEX/paper2/main.tex")

# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(figures_desc_file)
# Escribir el DataFrame a un archivo XLSX
df.to_excel('Excel_Desc2.xlsx', index=False)

Integrable systems and cluster algebras
Contenido completo:


In [ ]:
# Se llama a la función generadora de descripciónes
figures_desc_file = figures_descriptor("/media/Llama-MSVH/Archivos_TEX/paper3/subdivision_arxiv1.tex")
# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(figures_desc_file)
# Escribir el DataFrame a un archivo XLSX
df.to_excel('Excel_Desc3.xlsx', index=False)

Equi-dimensionalization via subdivision of simplices
Contenido completo:


In [ ]:
# Se llama a la función generadora de descripciónes
figures_desc_file = figures_descriptor("/media/Llama-MSVH/Archivos_TEX/paper4/Main.tex")
# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(figures_desc_file)
# Escribir el DataFrame a un archivo XLSX
df.to_excel('Excel_Desc4.xlsx', index=False)

A Tunable Reflection Surface with Independently Variable Phase and Slope
Contenido completo:


In [ ]:
import re

texto = r"""\begin{figure}[!h]
\centering
\tikzset{every picture/.style={scale=.6}, every node/.style={scale=.9}}
\input{Figures/RISConfiguration/Unit-cell}
\caption{The Configurations of the dog-bone and patch elements. The red rectangles are the positions of the varactors in the two elements. The dimensions considered in simulations are $L = 27$~mm, $W = 27$~mm, $l_{1} = 10$~mm, $w_{1} = 0.5$~mm, $g_{1} = 7$~mm, $l_{2} = 26$~mm, and $w_{2} = 20$~mm. The substrate is Rogers 5880 with a thickness of $1.57$~mm.}
\label{UnitCell}
\vspace{-0.25cm}
\end{figure}"""

figures = re.findall(r'\\begin\{figure\}(.*?)\\end\{figure\}', texto, re.DOTALL)

print(figures)

['[!h]\n\\centering\n\\tikzset{every picture/.style={scale=.6}, every node/.style={scale=.9}}\n\\input{Figures/RISConfiguration/Unit-cell}\n\\caption{The Configurations of the dog-bone and patch elements. The red rectangles are the positions of the varactors in the two elements. The dimensions considered in simulations are $L = 27$~mm, $W = 27$~mm, $l_{1} = 10$~mm, $w_{1} = 0.5$~mm, $g_{1} = 7$~mm, $l_{2} = 26$~mm, and $w_{2} = 20$~mm. The substrate is Rogers 5880 with a thickness of $1.57$~mm.}\n\\label{UnitCell}\n\\vspace{-0.25cm}\n']


In [ ]:
prompt = "What are you?"
response = generate_text_basic(prompt)

print(response)

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc74d668dc0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
print(conversation_history)

[]


In [ ]:
!curl localhost:5000

curl: (7) Failed to connect to localhost port 5000 after 0 ms: Connection refused
